---
title: 文本向量系列-如何基于拼音构建字向量
date: 2018-08-17 12:17:55
tags: [python, 文本挖掘]
toc: true
xiongzhang: true
xiongzhang_images: [main.jpg]

---
<span></span>
<!-- more -->


### 系列文章

这个是系列博客, 所有文章链接都列在这里, 并持续更新中。

<ul>
<li><a href="http://mlln.cn/2018/08/17/文本向量化理论和实践教程/" >文本向量化理论和实践教程</a></li>
<li><a href="http://mlln.cn/2018/08/17/文本向量系列-如何基于拼音构建字向量/" >文本向量系列-如何基于拼音构建字向量</a></li>
<li><a href="http://mlln.cn/2018/08/18/文本向量系列-如何基于笔画+拼音构建字向量/" >文本向量系列-如何基于笔画+拼音构建字向量/</a></li>
<li><a href="http://mlln.cn/2018/08/18/文本向量系列-如何基于词频矩阵和TF-IDF权重构建词向量/" >文本向量系列-如何基于词频矩阵和TF-IDF权重构建词向量/</a></li>
</ul>

用拼音来表示字向量的方法我见的比较少, 也没有见过类似的研究, 我这里只是介绍一下这个方法, 但是并不能评价这个方法的好坏. 在一些英文语料的研究中见过一些研究者使用字母来形成词向量的, 因为总共有26个英文字母, 所以一个词/拼音可以表示为26维的字母频率向量。

我们使用的开发环境:

- jupyter notebook
- python3.6

### 算法

主要包括两个步骤:

- 汉字转拼音
- 获取唯一字符集
- 拼音向量化

### 使用pypinyin工具进行汉字转拼音


安装方法:

```
pip install pypinyin
```

#### 基本用法

In [3]:
from pypinyin import pinyin
pinyin('我爱你中国')

[['wǒ'], ['ài'], ['nǐ'], ['zhōng'], ['guó']]

### 使用测试语料库

我们可以看到, 输出的是带有声调的拼音, 假设我不知道总共有多少种不同的字符构成了拼音(实际上是可以算出来的), 因为很多情况下我们都是根据语料库来确定的, 只要字符数量足够大能够表征我们的语料库就可以了, 所以我们需要一个语料库。

我们假设使用下面的语料库:

```
随着互联网, 尤其是移动互联网的快速发展, 社交媒体已经变成人们交流和传递思想的主要平台。每天从社交平台上产生的信息交互量难以估计。面对如此海量的消息, 如何进行科学的有效管理, 已成为当前的研究热点。根据中国互联网发展统计报告, 文本信息已占网络资源的70%以上, 是互联网中信息传播的主要载体, 每天从网络中产生的文本信息量在TB级别以上。在文本处理领域, 一般将文本信息分为长文本和短文本。互联网产生的数据大多数以短文本为主, 如腾讯空间说说、新浪微博、百度知识问答和淘宝商品的评价等。相对于传统大篇幅的长文本, 人们更容易接受以短文本的形式进行交流。如新浪微博限制其消息字数为140, 知识问答都是以简略语句的形式来概括。短文本能更好的表达出人们的情绪, 人们更喜欢以几个字甚至一个表情来表达自己的情感倾向。如何使用机器学习的相关方法对这些数据进行分析, 挖掘出有用的信息, 从而更好的利用互联网改善人民的生活已经变得日趋重要, 如帮助商家提供决策, 以使其利益最大化, 帮助用户更有效的提高产品体验, 是当前文本挖掘的重要课题。
```
注意, 我们的语料库还包括标点符号和数字, 我们把这些符号也当作"字"来进行向量化。

### 获取所有不同的字符(字符集)

在文本处理的工作中, 经常会用到词典(语料库包含的所有词的集合), 我们现在也要先找到这个"词典", 姑且就叫他字符集。每个字符对应一个整数作为他的ID。"字典"的作用就是可以将字符/词与数字进行互转。

In [11]:
from itertools import chain
from collections import Counter

text = '''随着互联网, 尤其是移动互联网的快速发展, 社交媒体已经变成人们交流和传递思想的主要平台。每天从社交平台上产生的信息交互量难以估计。面对如此海量的消息, 如何进行科学的有效管理, 已成为当前的研究热点。根据中国互联网发展统计报告, 文本信息已占网络资源的70%以上, 是互联网中信息传播的主要载体, 每天从网络中产生的文本信息量在TB级别以上。在文本处理领域, 一般将文本信息分为长文本和短文本。互联网产生的数据大多数以短文本为主, 如腾讯空间说说、新浪微博、百度知识问答和淘宝商品的评价等。相对于传统大篇幅的长文本, 人们更容易接受以短文本的形式进行交流。如新浪微博限制其消息字数为140, 知识问答都是以简略语句的形式来概括。短文本能更好的表达出人们的情绪, 人们更喜欢以几个字甚至一个表情来表达自己的情感倾向。如何使用机器学习的相关方法对这些数据进行分析, 挖掘出有用的信息, 从而更好的利用互联网改善人民的生活已经变得日趋重要, 如帮助商家提供决策, 以使其利益最大化, 帮助用户更有效的提高产品体验, 是当前文本挖掘的重要课题。'''
zi = chain(*pinyin(text))
chars = chain(*zi)
char_counts = Counter(chars)
print(char_counts)

Counter({'n': 173, 'g': 95, 'h': 82, 'i': 67, 'd': 47, 'x': 38, 'u': 37, 'é': 37, 'ǎ': 35, 'y': 35, 'ì': 34, 'à': 34, 's': 33, 'ā': 30, 'j': 30, 'z': 29, 'w': 29, 'í': 27, 'e': 26, 'ǐ': 26, 'b': 26, 'ù': 23, 'l': 23, 'á': 23, 'o': 23, 'ī': 19, ',': 18, ' ': 18, 'è': 18, 'c': 17, 'r': 15, 't': 14, 'ě': 14, 'ē': 12, 'q': 11, 'ò': 11, 'ú': 10, '。': 10, 'ō': 10, 'ó': 9, 'm': 9, 'f': 7, 'p': 6, 'k': 5, 'ǔ': 5, 'ū': 5, 'ǒ': 5, '0': 2, '、': 2, '7': 1, '%': 1, 'T': 1, 'B': 1, '1': 1, '4': 1, 'ü': 1})


In [12]:
# 求所有的字符
unique_chars = sorted(char_counts.keys())
print(unique_chars)

[' ', '%', ',', '0', '1', '4', '7', 'B', 'T', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'x', 'y', 'z', 'à', 'á', 'è', 'é', 'ì', 'í', 'ò', 'ó', 'ù', 'ú', 'ü', 'ā', 'ē', 'ě', 'ī', 'ō', 'ū', 'ǎ', 'ǐ', 'ǒ', 'ǔ', '、', '。']


In [14]:
# 构建词典
dictionary = dict(zip(unique_chars, range(len(unique_chars))))
dictionary

{' ': 0,
 '%': 1,
 ',': 2,
 '0': 3,
 '1': 4,
 '4': 5,
 '7': 6,
 'B': 7,
 'T': 8,
 'b': 9,
 'c': 10,
 'd': 11,
 'e': 12,
 'f': 13,
 'g': 14,
 'h': 15,
 'i': 16,
 'j': 17,
 'k': 18,
 'l': 19,
 'm': 20,
 'n': 21,
 'o': 22,
 'p': 23,
 'q': 24,
 'r': 25,
 's': 26,
 't': 27,
 'u': 28,
 'w': 29,
 'x': 30,
 'y': 31,
 'z': 32,
 'à': 33,
 'á': 34,
 'è': 35,
 'é': 36,
 'ì': 37,
 'í': 38,
 'ò': 39,
 'ó': 40,
 'ù': 41,
 'ú': 42,
 'ü': 43,
 'ā': 44,
 'ē': 45,
 'ě': 46,
 'ī': 47,
 'ō': 48,
 'ū': 49,
 'ǎ': 50,
 'ǐ': 51,
 'ǒ': 52,
 'ǔ': 53,
 '、': 54,
 '。': 55}

### 计算字向量

现在我们已经准备好了所有形成字向量需要用到的基本条件, 下面我们开始:

In [22]:
import numpy as np
def vectorize(word, dictionary):
    # 字转拼音
    chars = pinyin(word)[0][0]
    # 统计拼音中出现的字符个数
    counts = Counter(chars)
    # 构建一个0向量
    vector = np.zeros((len(dictionary)))
    # 使用字符频率给向量元素赋值
    for char, cnt in counts.items():
        index = dictionary[char]
        vector[index]=cnt
    return vector
    
vectorize('长', dictionary)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0.])

### 总结

这篇教程的主要用到了汉字转拼音的模块: pypinyin, 不过只用到了这个模块的其中一个功能。

字拼音向量化的思想主要是统计每个可能字符的频率, 所有字符的频率按照顺序组合到一起就构成了字的向量。

作为实战的第一篇, 这个教程比较简单啦, 只要python入门了就能看懂, 后面可能会逐步增加难度。